In [1]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from starter_code.visualize import visualize
from starter_code.utils import load_case

import tensorflow as tf

In [2]:
volume, segmentation = load_case(123)
X = volume.get_data()
y = segmentation.get_data()

IMG_WIDTH = X.shape[2]
IMG_HEIGHT = X.shape[1]
IMG_SLICES = X.shape[0]
# X = np.expand_dims(X, 3)


# y = tf.keras.utils.to_categorical(y)

In [3]:
from NeuralNetwork import *
model = makeModel(IMG_WIDTH, IMG_HEIGHT, 1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
checkpointer = tf.keras.callbacks.ModelCheckpoint('model1.h5', verbose=1, save_best_only=True)

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='accuracy'),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]


In [5]:
for n_img in np.arange(0, 10, 1):
    volume, segmentation = load_case(n_img)
    X = volume.get_data()
    y = segmentation.get_data()
    X = np.expand_dims(X, 3)
    y = tf.keras.utils.to_categorical(y)
    results = model.fit(X, y, batch_size=16, epochs=4, callbacks=callbacks, verbose=1)

Train on 611 samples
Epoch 1/4
611/611 [==============================] - 30s 49ms/sample - loss: 0.7871 - accuracy: 0.8694
Epoch 2/4
611/611 [==============================] - 23s 38ms/sample - loss: 0.0234 - accuracy: 0.9941
Epoch 3/4
611/611 [==============================] - 23s 38ms/sample - loss: 0.0149 - accuracy: 0.9941
Epoch 4/4
611/611 [==============================] - 23s 38ms/sample - loss: 0.0112 - accuracy: 0.9941
Train on 602 samples
Epoch 1/4
602/602 [==============================] - 26s 42ms/sample - loss: 0.0169 - accuracy: 0.9904
Epoch 2/4
602/602 [==============================] - 23s 38ms/sample - loss: 0.0124 - accuracy: 0.9904
Epoch 3/4
602/602 [==============================] - 23s 38ms/sample - loss: 0.0116 - accuracy: 0.9905
Epoch 4/4
602/602 [==============================] - 23s 38ms/sample - loss: 0.0110 - accuracy: 0.9906
Train on 261 samples
Epoch 1/4
261/261 [==============================] - 12s 46ms/sample - loss: 0.0408 - accuracy: 0.9902
Epoch 2/4


In [6]:
predicted = model.predict(X)

In [7]:
print(np.max(predicted[:,:,:,]))

0.28460273


In [18]:
def dice_score(y_pred, y_test):
    return np.sum(y_pred)*2.0 / (np.sum(y_pred) + np.sum(y_test))

In [23]:
dice_score(predicted[:,:,:,0], y[:,:,:,0])

1.0000662270509035

In [24]:
%matplotlib qt
from visualizeSlider import *
cube_show_slider(cube=predicted[:,:,:,1], axis=0, cmap='gray')